In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant import pipeline
from llm_data_quality_assistant.corruptor import RowCorruptionTypes, CellCorruptionTypes
from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np

load_dotenv()


True

In [5]:
corrupt_dataset = pd.read_csv("../datasets/parker_datasets/SHORT_allergen.csv")

gold_standard = pd.read_csv("../datasets/parker_datasets/gold_standard_alergene_pivoted.csv")

print(corrupt_dataset.head(2))
print(gold_standard.head(2))
print(type(gold_standard.get("code").iloc[0]))
print(type(corrupt_dataset.get("code").iloc[0]))

            code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0  4104420005280     2           0            0               0         2   
1  4104420005280     2           0            0               0         0   

   pistachio  walnut  cashew  celery  ...  fish  gluten  lupin  milk  \
0          0       0       0       0  ...     0       2      2     1   
1          0       0       0       0  ...     0       2      2     1   

   molluscs  mustard  peanut  sesame  soy  sulfite  
0         0        0       0       0    1        0  
1         0        0       0       0    1        0  

[2 rows x 22 columns]
            code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0  4104420006065     0           0            0               0         0   
1  4104420006065     0           0            0               0         0   

   pistachio  walnut  cashew  celery  ...  fish  gluten  lupin  milk  \
0          0       0       0       0  ...     0       2      0   

In [ ]:


shortened_corrupt_df = corrupt_dataset[corrupt_dataset["code"].isin(gold_standard["code"])]

print("Shape shortened corrupt dataset:")
print(shortened_corrupt_df.shape)

shortened_gold_standard = gold_standard[gold_standard["code"].isin(shortened_corrupt_df["code"])]

p = pipeline.Pipeline(shortened_gold_standard)

print("Shape shortened gold standard dataset:")
print(shortened_gold_standard.shape)

assert shortened_gold_standard.shape == shortened_corrupt_df.shape

# Find coordinates where entries differ
diff = shortened_gold_standard.values != shortened_corrupt_df.values
corrupted_coords = np.argwhere(diff)

print("Corrupted coordinates (row, col):")
print(corrupted_coords)

merged_df = p.clean_single_dataset(shortened_corrupt_df)

shortened_merged_df = merged_df[merged_df["code"].isin(gold_standard["code"])]

print("Shape merged dataset:")
print(merged_df.shape)

print("Shape shortened merged dataset:")
print(shortened_merged_df.shape)

results = p.evaluate_micro(shortened_merged_df, [corrupted_coords])
print(results)

same_values_only = (shortened_merged_df.values == shortened_corrupt_df.values).all()
print(same_values_only)


            code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0  4104420005280     2           0            0               0         2   
1  4104420005280     2           0            0               0         0   
2  4104420005488     1           0            0               0         0   
3  4104420005488     0           0            0               0         0   
4  4104420006065     0           0            0               0         0   
5  4104420006065     0           0            0               0         0   
6  4104420007840     0           0            0               0         0   
7  4104420007840     0           0            0               0         0   
8  4104420007901     0           0            0               0         0   
9  4104420007901     0           0            0               0         0   

   pistachio  walnut  cashew  celery  ...  fish  gluten  lupin  milk  \
0          0       0       0       0  ...     0       2      2     1   
1       

: 